## Setup

In [5]:
! pip install openai python-dotenv elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.6/524.6 kB 5.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [34]:
from openai import OpenAI
from elasticsearch import Elasticsearch
import json
from tqdm.auto import tqdm 

In [35]:
client = OpenAI(
    base_url = 'http://localhost:11434/v1/',#
    api_key = 'ollama'
)

In [36]:
q = 'The course has already started, can I still enroll?'

## Simple RAG

In [34]:
prompt_template = """
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ databse. 
Use only the facts in the CONTEXT when answering the question.
If the context doesn't contain the answer, output NONE.

QUESTION: 
{question}

CONTEXT: 
{context}
""".strip()

In [35]:
context = ""

for doc in results:
    context += f"section: {doc['section']} \nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [31]:
print(context)

section: General course-related questions 
question: The course has already started. Can I still join it?
answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section: General course-related questions 
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions 
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so yo

In [38]:
prompt = prompt_template.format(question=q, context = context).strip()

## Modularization

In [37]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    results = index.search(
        query = query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )   
    return results



In [38]:
#search("how do I run kafka?")

In [39]:
def build_prompt(query, search_results):
    prompt_template = """
    You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ databse. 
    Use only the facts in the CONTEXT when answering the question.

    QUESTION: 
    {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""

    for doc in search_results:
        context += f"section: {doc['section']} \nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context = context).strip()

    return prompt

In [40]:
def llm(prompt):
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [41]:
query = "how do I run kafka?"
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

In [42]:
print(answer)

NameError: name 'answer' is not defined

## Using elastic search

Command to run elastic search  
docker run -it --name elasticsearch -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" -e "xpack.security.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.4.3

http get request to local host  
curl http://localhost:9200

In [53]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)
    
documents = []

#docs_raw[2] # 3 courses
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [54]:
es_client = Elasticsearch('http://localhost:9200')
es_client.info()


ObjectApiResponse({'name': 'e79cd6cdd621', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'iovZK3DNTmmzBYF-RohZIw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [55]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

In [56]:
index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [57]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:28<00:00, 33.50it/s]


In [58]:
query = 'I just discovered the course, can I still join it?'

In [59]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs


In [49]:
results = elastic_search(query)

In [60]:
def build_prompt(query, search_results):
    prompt_template = """
    You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ databse. 
    Use only the facts in the CONTEXT when answering the question.

    QUESTION: 
    {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""

    for doc in search_results:
        context += f"section: {doc['section']} \nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context = context).strip()

    return prompt
def llm(prompt):
    response = client.chat.completions.create(
        model = 'phi3',
        messages = [{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [61]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
response = rag(query)

In [ ]:
response